# API 資料串接 - 以 知乎 API 實作範例



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭




## 作業目標

* 根據範例提供的 API ，完成以下問題：

    1. 取出知乎問題發問時間
    2. 取出第一筆與最後一筆回答的時間


### 1. 取出知乎問題發問時間

In [1]:
#個人備註:
#JsonPath與Xpath語法對比
#Json結構清晰，可讀性高，複雜度低，非常容易匹配，下表中對應了XPath的用法
#之前用過一點xpath，找的那個路徑簡直是頭暈頭花
#JsonPath 是一種資訊抽取類庫，是從JSON文件中抽取指定資訊的工具
#以下是JsonPath與Xpath語法對比
#======================================================================

import pandas as pd


a=['/','.','/','..','//','*','@','[]','blank','[]','n/a','()']
b=['$','@','.or[]','n/a','..','*','n/a','[]','[,]','?()','()','n/a']
c=['根節點','現行節點','取子節點','取父節點，Jsonpath未支援','就是不管位置，選擇所有符合條件的條件','匹配所有元素節點','根據屬性訪問，Json不支援，因為Json是個Key-value遞迴結構，不需要。','迭代器標示（可以在裡邊做簡單的迭代操作，如陣列下標，根據內容選值等）','支援迭代器中做多選。','支援過濾操作.','支援表示式計算','分組，JsonPath不支援']
d={"XPath" : a,
   "JSONPath" : b,
   "描述" : c}
data=pd.DataFrame(d)
data

#來源:https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/507102/


,XPath,JSONPath,描述
0,/,$,根節點
1,.,@,現行節點
2,/,.or[],取子節點
3,..,n/a,取父節點，Jsonpath未支援
4,//,..,就是不管位置，選擇所有符合條件的條件
5,*,*,匹配所有元素節點
6,@,n/a,根據屬性訪問，Json不支援，因為Json是個Key-value遞迴結構，不需要。
7,[],[],迭代器標示（可以在裡邊做簡單的迭代操作，如陣列下標，根據內容選值等）
8,blank,"[,]",支援迭代器中做多選。
9,[],?(),支援過濾操作.


In [2]:
# 1. 取出知乎問題發問時間

#個人備註:
#知乎伺服器只會檢查 user-agent 而已，如果在
#一開始不確定的情況下，會建議所有參參數都加上去！
#通常很多網站都只檢查user-agent
#======================================================================

import requests
import json

headers={'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Mobile Safari/537.36'}

r = requests.get('https://www.zhihu.com/api/v4/questions/55493026/answers',headers = headers)

js = json.loads(r.text)
js

{'data': [{'id': 683070334,
   'type': 'answer',
   'answer_type': 'normal',
   'question': {'type': 'question',
    'id': 55493026,
    'title': '你们都是怎么学 Python 的？',
    'question_type': 'normal',
    'created': 1486390229,
    'updated_time': 1582533957,
    'url': 'https://www.zhihu.com/api/v4/questions/55493026',
    'relationship': {}},
   'author': {'id': '36f69162230003d316d0b8a6d8da20ba',
    'url_token': 'liang-zi-wei-48',
    'name': '量子位',
    'avatar_url': 'https://pic4.zhimg.com/v2-ca6e7ffc10a0d10edbae635cee82d007_is.jpg',
    'avatar_url_template': 'https://pic4.zhimg.com/v2-ca6e7ffc10a0d10edbae635cee82d007_{size}.jpg',
    'is_org': True,
    'type': 'people',
    'url': 'https://www.zhihu.com/api/v4/people/36f69162230003d316d0b8a6d8da20ba',
    'user_type': 'organization',
    'headline': '有趣的AI&amp;前沿科技→_→ 公众号：QbitAI',
    'badge': [{'type': 'identity', 'description': '已认证的官方帐号', 'topics': []},
     {'type': 'best_answerer', 'description': '优秀回答者', 'topics': []}],
    

In [3]:
#個人備註:
#Python 的時間還有時區處理
#https://kkc.github.io/2015/07/08/dealing-with-datetime-and-timezone-in-python/
#from datetime import datetime

#datetime裡的套件fromtimestamp
#datetime.fromtimestamp

#======================================================================
import jsonpath as jp
from datetime import datetime



time = jp.jsonpath(js,'$..created')
for i in time:
    print(datetime.fromtimestamp(i))
#從json的資訊結構去分析，估計這5個時間都是代表著問題的'發問時間'。

2017-02-06 22:10:29
2017-02-06 22:10:29
2017-02-06 22:10:29
2017-02-06 22:10:29
2017-02-06 22:10:29


# 答:

1.取出知乎問題發問時間

從json的資訊結構去分析，估計這5個時間都是代表著問題的'發問時間'。

即2017-02-06 22:10:29

### 2. 取出第一筆與最後一筆回答的時間

In [4]:
# 2. 取出第一筆與最後一筆回答的時間

#個人備註:
#調動第0筆與第4筆，即他回傳的5項資料中的頭與尾
#======================================================================

df=datetime.fromtimestamp

first_time = jp.jsonpath(js,'$..created_time')[0]
last_time = jp.jsonpath(js,'$..created_time')[4]

print('答:\n')
print('\t第一筆回答的時間:',df(first_time))
print('\t最後一筆回答的時間:',df(last_time))

答:

	第一筆回答的時間: 2019-05-14 17:00:12
	最後一筆回答的時間: 2019-02-27 11:41:25
